In [1]:
import datetime
import logging
import time
from pathlib import Path

In [2]:
import pandas as pd

In [3]:
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.utils.export import generate_multimodal_pages
from docling.utils.utils import create_hash

/home/himanshu/ibm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
_log = logging.getLogger(__name__)

In [5]:
IMAGE_RESOLUTION_SCALE = 2.0

In [6]:
def main():
    logging.basicConfig(level=logging.INFO)

    input_doc_path = Path("paper.pdf")
    output_dir = Path("scratch")

    # Important: For operating with page images, we must keep them, otherwise the DocumentConverter
    # will destroy them for cleaning up memory.
    # This is done by setting AssembleOptions.images_scale, which also defines the scale of images.
    # scale=1 correspond of a standard 72 DPI image
    pipeline_options = PdfPipelineOptions()
    pipeline_options.images_scale = IMAGE_RESOLUTION_SCALE
    pipeline_options.generate_page_images = True

    doc_converter = DocumentConverter(
        format_options={
            InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
        }
    )

    start_time = time.time()

    conv_res = doc_converter.convert(input_doc_path)

    output_dir.mkdir(parents=True, exist_ok=True)

    rows = []
    for (
        content_text,
        content_md,
        content_dt,
        page_cells,
        page_segments,
        page,
    ) in generate_multimodal_pages(conv_res):

        dpi = page._default_image_scale * 72

        rows.append(
            {
                "document": conv_res.input.file.name,
                "hash": conv_res.input.document_hash,
                "page_hash": create_hash(
                    conv_res.input.document_hash + ":" + str(page.page_no - 1)
                ),
                "image": {
                    "width": page.image.width,
                    "height": page.image.height,
                    "bytes": page.image.tobytes(),
                },
                "cells": page_cells,
                "contents": content_text,
                "contents_md": content_md,
                "contents_dt": content_dt,
                "segments": page_segments,
                "extra": {
                    "page_num": page.page_no + 1,
                    "width_in_points": page.size.width,
                    "height_in_points": page.size.height,
                    "dpi": dpi,
                },
            }
        )

    # Generate one parquet from all documents
    df = pd.json_normalize(rows)
    now = datetime.datetime.now()
    output_filename = output_dir / f"multimodal_{now:%Y-%m-%d_%H%M%S}.parquet"
    df.to_parquet(output_filename)

    end_time = time.time() - start_time

    _log.info(
        f"Document converted and multimodal pages generated in {end_time:.2f} seconds."
    )

    # This block demonstrates how the file can be opened with the HF datasets library
    # from datasets import Dataset
    # from PIL import Image
    # multimodal_df = pd.read_parquet(output_filename)

    # # Convert pandas DataFrame to Hugging Face Dataset and load bytes into image
    # dataset = Dataset.from_pandas(multimodal_df)
    # def transforms(examples):
    #     examples["image"] = Image.frombytes('RGB', (examples["image.width"], examples["image.height"]), examples["image.bytes"], 'raw')
    #     return examples
    # dataset = dataset.map(transforms)

In [7]:
main()

INFO:docling.document_converter:Going to convert document batch...


INFO:docling.utils.accelerator_utils:Accelerator device: 'cuda:0'
INFO:docling.utils.accelerator_utils:Accelerator device: 'cuda:0'
/home/himanshu/ibm/lib/python3.10/site-packages/torch/utils/cpp_extension.py:2059: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Could not load the custom kernel for multi-scale deformable attention: Error building extension 'MultiScaleDeformableAttention': [1/2] /usr/local/cuda/bin/nvcc --generate-dependencies-with-compile --dependency-output ms_deform_attn_cuda.cuda.o.d -DTORCH_EXTENSION_NAME=MultiScaleDeformableAttention -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1016\" -I/home/himanshu/ibm/lib/python3.10/site-packages/transformers/kernels/deformable_detr -isystem /home/himanshu/ibm/lib/python3.10/site-packages/torch/include

In [8]:
from datasets import Dataset
from PIL import Image
multimodal_df = pd.read_parquet("./scratch/multimodal_2025-02-13_080455.parquet")

# Convert pandas DataFrame to Hugging Face Dataset and load bytes into image
dataset = Dataset.from_pandas(multimodal_df)
def transforms(examples):
    examples["image"] = Image.frombytes('RGB', (examples["image.width"], examples["image.height"]), examples["image.bytes"], 'raw')
    return examples
dataset = dataset.map(transforms)

INFO:datasets:PyTorch version 2.6.0+cu126 available.


Map: 100%|██████████| 31/31 [00:02<00:00, 11.53 examples/s] 


In [18]:
print(dataset)

Dataset({
    features: ['document', 'hash', 'page_hash', 'cells', 'contents', 'contents_md', 'contents_dt', 'segments', 'image.width', 'image.height', 'image.bytes', 'extra.page_num', 'extra.width_in_points', 'extra.height_in_points', 'extra.dpi', 'image'],
    num_rows: 31
})


In [13]:
dataset['segments']

[[{'bbox': [0.16898855820200803,
    0.12997351752387154,
    0.8310016307955475,
    0.16851390491832388],
   'data': [],
   'index_in_doc': 0,
   'label': 'section_header',
   'text': 'FastVPINNs: Tensor-Driven Acceleration of VPINNs for Complex Geometries'},
  {'bbox': [0.14705882352941177,
    0.19446212121212125,
    0.29790522875816994,
    0.20897601010101013],
   'data': [],
   'index_in_doc': 1,
   'label': 'text',
   'text': 'Thivin Anandh 1'},
  {'bbox': [0.6578676470588236,
    0.19986868686868695,
    0.8529477124183007,
    0.20633459595959602],
   'data': [],
   'index_in_doc': 2,
   'label': 'text',
   'text': 'thivinanandh@iisc.ac.in'},
  {'bbox': [0.14705882352941177,
    0.21156944444444445,
    0.2670718954248366,
    0.22608333333333333],
   'data': [],
   'index_in_doc': 3,
   'label': 'text',
   'text': 'Divij Ghose 1'},
  {'bbox': [0.683171568627451,
    0.21697601010101014,
    0.8529346405228758,
    0.22344191919191922],
   'data': [],
   'index_in_doc': 4,
 